# Sample Notebook for Zero-Shot Inference with CheXzero
This notebook walks through how to use CheXzero to perform zero-shot inference on a chest x-ray image dataset.

## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple, Optional

import sys
sys.path.append('../')

from eval import evaluate, bootstrap
from zero_shot import make, make_true_labels, run_softmax_eval

%load_ext autoreload
%autoreload 2

## Directories and Constants

In [38]:
## Define Zero Shot Labels and Templates

# ----- DIRECTORIES ------ #
# # Padchest
# cxr_filepath: str = '/home/ec2-user/all_raw_data/padchest/images/44_cxr.h5' # filepath of chest x-ray images (.h5)
# cxr_true_labels_path: Optional[str] = '/home/ec2-user/all_raw_data/padchest/44_cxr_labels.csv' # (optional for evaluation) if labels are provided, provide path

# # CheXzero test
# cxr_filepath: str = '/home/ec2-user/CHEXLOCALIZE/CheXpert/test.h5' # filepath of chest x-ray images (.h5)
# cxr_true_labels_path: Optional[str] = '/home/ec2-user/CHEXLOCALIZE/CheXpert/test_labels_view1.csv' # (optional for evaluation) if labels are provided, provide path

# CheXzero val
cxr_filepath: str = '/home/ec2-user/all_raw_data/chexpert/CheXpert-v1.0-small/valid/chexpert_val.h5' # filepath of chest x-ray images (.h5)
cxr_true_labels_path: Optional[str] = '/home/ec2-user/all_raw_data/chexpert/CheXpert-v1.0-small/valid_view1.csv' # (optional for evaluation) if labels are provided, provide path


model_dir: str = '../checkpoints/pranav' # where pretrained models are saved (.pt) 
predictions_dir: Path = Path('../predictions') # where to save predictions
cache_dir: str = predictions_dir / "cached" # where to cache ensembled predictions

context_length: int = 77

# ------- LABELS ------  #
# Define labels to query each image | will return a prediction for each label
# cxr_labels: List[str] = ['Atelectasis','Cardiomegaly', 
#                                       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
#                                       'Lung Opacity', 'No Finding','Pleural Effusion', 'Pleural Other', 'Pneumonia', 
#                                       'Pneumothorax', 'Support Devices']
cxr_labels = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
                'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
                'Fracture', 'Support Devices']
# cxr_labels = [label.lower() for label in cxr_labels]

# ---- TEMPLATES ----- # 
# Define set of templates | see Figure 1 for more details                        
cxr_pair_template: Tuple[str] = ("{}", "no {}")

# ----- MODEL PATHS ------ #
# If using ensemble, collect all model paths
model_paths = []
for subdir, dirs, files in os.walk(model_dir):
    for file in files:
        full_dir = os.path.join(subdir, file)
        model_paths.append(full_dir)

n = len(model_paths)
# model_paths = model_paths[-3:]
print(model_paths)

['../checkpoints/cxr-bert/checkpoint_100000.pt', '../checkpoints/cxr-bert/checkpoint_105000.pt', '../checkpoints/cxr-bert/checkpoint.pt']


## Run Inference

In [32]:
## Run the model on the data set using ensembled models
def ensemble_models(
    model_paths: List[str], 
    cxr_filepath: str, 
    cxr_labels: List[str], 
    cxr_pair_template: Tuple[str], 
    cache_dir: str = None, 
    save_name: str = None,
    change_text_encoder: bool = False,
) -> Tuple[List[np.ndarray], np.ndarray]: 
    """
    Given a list of `model_paths`, ensemble model and return
    predictions. Caches predictions at `cache_dir` if location provided.

    Returns a list of each model's predictions and the averaged
    set of predictions.
    """

    predictions = []
    model_paths = sorted(model_paths) # ensure consistency of 
    for path in model_paths: # for each model
        model_name = Path(path).stem

        # load in model and `torch.DataLoader`
        model, loader = make(
            model_path=path, 
            cxr_filepath=cxr_filepath, 
            change_text_encoder=change_text_encoder,
        ) 
        
        # path to the cached prediction
        if cache_dir is not None:
            if save_name is not None: 
                cache_path = Path(cache_dir) / f"{save_name}_{model_name}.npy"
            else: 
                cache_path = Path(cache_dir) / f"{model_name}.npy"

        # if prediction already cached, don't recompute prediction
        if cache_dir is not None and os.path.exists(cache_path): 
            print("Loading cached prediction for {}".format(model_name))
            y_pred = np.load(cache_path)
        else: # cached prediction not found, compute preds
            print("Inferring model {}".format(path))
            y_pred = run_softmax_eval(model, loader, cxr_labels, cxr_pair_template, change_text_encoder=change_text_encoder)
            if cache_dir is not None: 
                Path(cache_dir).mkdir(exist_ok=True, parents=True)
                np.save(file=cache_path, arr=y_pred)
        predictions.append(y_pred)
    
    # compute average predictions
    if len(predictions) > 1:
        y_pred_avg = np.mean(predictions, axis=0)
    else:
        y_pred_avg = predictions[0]
    
    return predictions, y_pred_avg

In [36]:
# make test_true
test_true = make_true_labels(cxr_true_labels_path=cxr_true_labels_path, cxr_labels=cxr_labels)

result = []
# Evaluate all models alone
for path in model_paths:
    predictions, y_pred_avg = ensemble_models(
        model_paths=[path], 
        cxr_filepath=cxr_filepath, 
        cxr_labels=cxr_labels, 
        cxr_pair_template=cxr_pair_template, 
        cache_dir=cache_dir,
        change_text_encoder = True,
    )

    test_pred = y_pred_avg

    # evaluate model
    cxr_results = evaluate(test_pred, test_true, cxr_labels)

    # boostrap evaluations for 95% confidence intervals
    bootstrap_results = bootstrap(test_pred, test_true, cxr_labels)
    mAUC = np.mean(bootstrap_results[1][["Atelectasis_auc", "Cardiomegaly_auc", "Consolidation_auc", "Edema_auc", "Pleural Effusion_auc"]].iloc[0, :])

    print("Model:{},  AUC: {}".format(path, mAUC))
    result.append([path, mAUC])

print(result)
    

../checkpoints/cxr-bert/checkpoint_5000.pt using an online model  
Inferring model ../checkpoints/cxr-bert/checkpoint_5000.pt


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

Model:../checkpoints/cxr-bert/checkpoint_100000.pt,  AUC: 0.6112714285714287
../checkpoints/cxr-bert/checkpoint_105000.pt using an online model  
Loading cached prediction for checkpoint_105000


  0%|          | 0/1000 [00:00<?, ?it/s]

Model:../checkpoints/cxr-bert/checkpoint_105000.pt,  AUC: 0.5823785714285714
../checkpoints/cxr-bert/checkpoint.pt using an online model  
Loading cached prediction for checkpoint


  0%|          | 0/1000 [00:00<?, ?it/s]

Model:../checkpoints/cxr-bert/checkpoint.pt,  AUC: 0.5691285714285714
[['../checkpoints/cxr-bert/checkpoint_5000.pt', 0.6789], ['../checkpoints/cxr-bert/checkpoint_10000.pt', 0.6785428571428571], ['../checkpoints/cxr-bert/checkpoint_15000.pt', 0.6998714285714286], ['../checkpoints/cxr-bert/checkpoint_20000.pt', 0.684342857142857], ['../checkpoints/cxr-bert/checkpoint_25000.pt', 0.6879571428571428], ['../checkpoints/cxr-bert/checkpoint_30000.pt', 0.6807428571428573], ['../checkpoints/cxr-bert/checkpoint_35000.pt', 0.6876], ['../checkpoints/cxr-bert/checkpoint_40000.pt', 0.6862214285714285], ['../checkpoints/cxr-bert/checkpoint_45000.pt', 0.6858357142857142], ['../checkpoints/cxr-bert/checkpoint_50000.pt', 0.6802285714285715], ['../checkpoints/cxr-bert/checkpoint_55000.pt', 0.6801857142857145], ['../checkpoints/cxr-bert/checkpoint_60000.pt', 0.6858428571428571], ['../checkpoints/cxr-bert/checkpoint_65000.pt', 0.6687785714285713], ['../checkpoints/cxr-bert/checkpoint_70000.pt', 0.65977142

In [39]:
predictions, y_pred_avg = ensemble_models(
    model_paths=model_paths, 
    cxr_filepath=cxr_filepath, 
    cxr_labels=cxr_labels, 
    cxr_pair_template=cxr_pair_template, 
    cache_dir=cache_dir,
    change_text_encoder = True,
)

../checkpoints/cxr-bert/checkpoint.pt using an online model  
Loading cached prediction for checkpoint
../checkpoints/cxr-bert/checkpoint_100000.pt using an online model  
Loading cached prediction for checkpoint_100000
../checkpoints/cxr-bert/checkpoint_105000.pt using an online model  
Loading cached prediction for checkpoint_105000


In [7]:
# save averaged preds
pred_name = "chexpert_preds_bert.npy" # add name of preds
predictions_dir = predictions_dir / pred_name
np.save(file=predictions_dir, arr=y_pred_avg)

## (Optional) Evaluate Results
If ground truth labels are available, compute AUC on each pathology to evaluate the performance of the zero-shot model. 

In [40]:
# make test_true
test_pred = y_pred_avg
test_true = make_true_labels(cxr_true_labels_path=cxr_true_labels_path, cxr_labels=cxr_labels)

# evaluate model
cxr_results = evaluate(test_pred, test_true, cxr_labels)

# boostrap evaluations for 95% confidence intervals
bootstrap_results = bootstrap(test_pred, test_true, cxr_labels)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [48]:
# display AUC with confidence intervals
bootstrap_results[1][["Atelectasis_auc", "Cardiomegaly_auc", "Consolidation_auc", "Edema_auc", "Pleural Effusion_auc"]]

,Atelectasis_auc,Cardiomegaly_auc,Consolidation_auc,Edema_auc,Pleural Effusion_auc
mean,0.5341,0.7555,0.6241,0.7631,0.5203
lower,0.4791,0.7090,0.5144,0.7115,0.4555
upper,0.5873,0.7973,0.7251,0.8132,0.5840


In [49]:
# top 5 competition pathologies
print("Mean AUC: {}".format(
    np.mean(bootstrap_results[1][["Atelectasis_auc", "Cardiomegaly_auc", "Consolidation_auc", "Edema_auc", "Pleural Effusion_auc"]].iloc[0, :])
    ))

Mean AUC: 0.63942


In [11]:
pd.DataFrame(bootstrap_results[1]).to_csv('chexzero_chexpert.csv')